# Estimate Roughness from LCDB v5

In [11]:
import rioxarray
import pathlib
import geopandas
import shapely
import geoapis.vector
import dotenv
import os
import numpy
import xarray

In [12]:
crs = 2193
fixed_zo = 0.05
base_path = pathlib.Path(r"C:\Local\data\catchments\Waikanae\caches\roughness\land_cover")

In [13]:
x0=1767800; x1=1775500
y0=5469612; y1=5477770
bbox = geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x0,y0],[x0,y1],[x1,y1],[x1,y0]])], crs=crs)

### Load in DEM

In [14]:
dem = rioxarray.rioxarray.open_rasterio(base_path / ".." / "geofabrics" / "geofabric_4m_with_waterways.nc",
            masked=True,
            parse_coordinates=True,
        ).squeeze("band", drop=True)
if "data_source" in dem.keys():
    dem["data_source"] = dem.data_source.astype(numpy.float32)
if "lidar_source" in dem.keys():
    dem["lidar_source"] = dem.data_source.astype(numpy.float32)
if "z" in dem.keys():
    dem["z"] = dem.z.astype(numpy.float32)

### Map zo

In [15]:
dem["zo"] = xarray.ones_like(dem.z) * fixed_zo
dem.zo.rio.write_crs(crs, inplace=True)
dem.zo.rio.write_nodata(numpy.nan, encoded=True, inplace=True)

<xarray.DataArray 'zo' (y: 2040, x: 1925)>
array([[0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05],
       ...,
       [0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.05, ..., 0.05, 0.05, 0.05]], dtype=float32)
Coordinates:
  * x            (x) float64 1.768e+06 1.768e+06 ... 1.775e+06 1.776e+06
  * y            (y) float64 5.478e+06 5.478e+06 5.478e+06 ... 5.47e+06 5.47e+06
    spatial_ref  int32 0

In [16]:
zo = dem.drop(["data_source", "lidar_source", "z"])
zo.to_netcdf(base_path / f"zo_fixed_{fixed_zo}.nc")